# Creating a weather API

The website will need an API that can return a list of destinations on the specified CCAAs (or the whole country) on a given date based on the predicted weather.

It must meet the following criteria:

- Takes a date and derives the week of the year.
- Returns the 5 best destination on that date based on meteorological data.
- Filters the 5 destinations by the specified CCAAs.
- Spreads results between all selected CCAAs.

# Assigning a weather station to each destination

Since most destination towns don't have a weather station we'll be simply assigning them the closest one (for the moment). This will be done by creating a new dataframe containing all town IDs (one for each row) as well as the weekly index of the nearest weather station.

In [101]:
import pandas as pd
import haversine as hs
import folium
import random
import datetime

In [2]:
#Importing our towns dataframe.

towns = pd.read_csv('towns_2207_201.csv')

In [3]:
towns.head()

,ID,municipality,ccaa,province,municipality_inhabitants,geographic_area,radius,routes_number,routes_ids,mountain_passes_ids,coords,coords_MDB
0,884,Barcelona,Cataluña,Barcelona,1664182,100.764400,5.663411,4,"[1292, 1732, 6228, 8149]","[315, 323, 822, 388]","(41.38424664,2.17634927)","[2.17634927,41.38424664]"
1,7257,València,Comunitat Valenciana,Valencia,800215,139.268700,6.658115,9,"[528, 1469, 1472, 2478, 5225, 7040, 7231, 7734...","[112, 631, 773, 1092, 597, 533, 769, 52, 424, ...","(39.47534441,-0.37565717)","[-0.37565717,39.47534441]"
2,4547,Málaga,Andalucía,Málaga,578460,395.706900,11.223062,10,"[2933, 4541, 4546, 5035, 5997, 5998, 8379, 841...","[86, 108, 109, 649, 171, 426, 488, 688, 700, 1...","(36.72034267,-4.41997511)","[-4.41997511,36.72034267]"
3,4613,Murcia,Región de Murcia,Murcia,459403,885.114900,16.785117,7,"[691, 1691, 6089, 6769, 7049, 8099, 8196]","[11, 787, 1103, 1055, 1012, 978, 1001]","(37.98436361,-1.1285408)","[-1.1285408,37.98436361]"
4,151,Alicante,Comunitat Valenciana,Alicante,337482,201.265845,8.004046,9,"[621, 629, 647, 680, 1469, 3189, 8145, 9500, 9...","[956, 626, 643, 644, 429, 534, 802, 99, 1092, ...","(38.34548705,-0.4831832)","[-0.4831832,38.34548705]"


In [4]:
#Importing our weather station data.

meteo = pd.read_csv('meteo.csv')

In [5]:
meteo.head()

,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,...,w46,w47,w48,w49,w50,w51,w52,w53,name,coords
0,7.6,7.6,7.2,7.2,7.6,7.6,7.8,7.8,7.6,8.0,...,7.8,7.8,7.2,7.6,7.6,7.8,7.8,7.8,0002I,"(40.95806,0.87139)"
1,7.0,7.0,7.0,7.0,7.0,7.0,7.2,7.6,7.4,7.4,...,7.2,7.6,7.2,7.0,7.2,7.6,7.2,7.0,0009X,"(41.21389,0.96333)"
2,7.6,7.0,7.4,7.2,7.2,7.2,7.6,7.6,7.2,7.6,...,8.4,7.8,7.4,7.6,7.4,7.6,7.4,6.8,0016A,"(41.14972,1.17889)"
3,7.8,7.8,7.8,7.8,7.8,7.8,7.8,7.8,7.8,8.2,...,8.6,8.2,7.8,7.8,7.8,7.8,7.8,7.8,0042Y,"(41.12389,1.24917)"
4,7.0,7.0,7.0,5.6,6.8,6.8,7.2,7.2,6.8,7.2,...,7.6,6.6,7.2,7.2,6.8,7.2,7.0,6.8,0061X,"(41.41694,1.51917)"


In [6]:
#Creating a new dataframe with the same columns as meteo.

df = pd.DataFrame(columns=meteo.columns)

In [7]:
df.head()

,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,...,w46,w47,w48,w49,w50,w51,w52,w53,name,coords


In [8]:
#Adding new columns to hold town ID,  CCAA and coords.

df['town_ID'] = None
df['CCAA_ID'] = None
df['coords'] = None

In [9]:
#Let's popuate the town_ID column by using a list.

df['town_ID'] = towns['ID'].tolist()

In [10]:
#Doing the same with CCAA.

df['CCAA_ID'] = towns['ccaa'].tolist()

In [11]:
#Performing the same procedure with the coordinates.

df['coords'] = towns['coords'].tolist()

In [12]:
df.head()

,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,...,w48,w49,w50,w51,w52,w53,name,coords,town_ID,CCAA_ID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.38424664,2.17634927)",884,Cataluña
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(39.47534441,-0.37565717)",7257,Comunitat Valenciana
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(36.72034267,-4.41997511)",4547,Andalucía
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(37.98436361,-1.1285408)",4613,Región de Murcia
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(38.34548705,-0.4831832)",151,Comunitat Valenciana


In [13]:
df['CCAA_ID'].value_counts()

Andalucía               44
Comunitat Valenciana    27
Galicia                 22
Castilla y León         21
País Vasco              20
Asturias                13
Cantabria               10
Cataluña                 9
Aragón                   8
Navarra                  7
Extremadura              5
Comunidad de Madrid      5
Castilla-La Mancha       4
Región de Murcia         3
La Rioja                 2
Name: CCAA_ID, dtype: int64

In [14]:
#Our next step will be turning each CCAA name into its ID this will have to be performed with a loop.

for i in range(len(df)):
    if df['CCAA_ID'].iloc[i] == 'Andalucía':
        df['CCAA_ID'].iloc[i] = '1'
    elif df['CCAA_ID'].iloc[i] == 'Comunitat Valenciana':
        df['CCAA_ID'].iloc[i] = '10'
    elif df['CCAA_ID'].iloc[i] == 'Galicia':
        df['CCAA_ID'].iloc[i] = '12'
    elif df['CCAA_ID'].iloc[i] == 'Castilla y León':
        df['CCAA_ID'].iloc[i] = '7'
    elif df['CCAA_ID'].iloc[i] == 'País Vasco':
        df['CCAA_ID'].iloc[i] = '16'
    elif df['CCAA_ID'].iloc[i] == 'Asturias':
        df['CCAA_ID'].iloc[i] = '3'
    elif df['CCAA_ID'].iloc[i] == 'Cantabria':
        df['CCAA_ID'].iloc[i] = '6'
    elif df['CCAA_ID'].iloc[i] == 'Cataluña':
        df['CCAA_ID'].iloc[i] = '9'
    elif df['CCAA_ID'].iloc[i] == 'Aragón':
        df['CCAA_ID'].iloc[i] = '2'
    elif df['CCAA_ID'].iloc[i] == 'Navarra':
        df['CCAA_ID'].iloc[i] = '15'
    elif df['CCAA_ID'].iloc[i] == 'Comunidad de Madrid':
        df['CCAA_ID'].iloc[i] = '13'
    elif df['CCAA_ID'].iloc[i] == 'Extremadura':
        df['CCAA_ID'].iloc[i] = '11'
    elif df['CCAA_ID'].iloc[i] == 'Castilla-La Mancha':
        df['CCAA_ID'].iloc[i] = '8'
    elif df['CCAA_ID'].iloc[i] == 'Región de Murcia':
        df['CCAA_ID'].iloc[i] = '14'
    elif df['CCAA_ID'].iloc[i] == 'La Rioja':
        df['CCAA_ID'].iloc[i] = '17'

C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [46]:
#Dropping 'name', we won't really use it.

meteo.drop('name', axis=1, inplace=True)
df.drop('name', axis=1, inplace=True)

In [15]:
meteo.head()

,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,...,w46,w47,w48,w49,w50,w51,w52,w53,name,coords
0,7.6,7.6,7.2,7.2,7.6,7.6,7.8,7.8,7.6,8.0,...,7.8,7.8,7.2,7.6,7.6,7.8,7.8,7.8,0002I,"(40.95806,0.87139)"
1,7.0,7.0,7.0,7.0,7.0,7.0,7.2,7.6,7.4,7.4,...,7.2,7.6,7.2,7.0,7.2,7.6,7.2,7.0,0009X,"(41.21389,0.96333)"
2,7.6,7.0,7.4,7.2,7.2,7.2,7.6,7.6,7.2,7.6,...,8.4,7.8,7.4,7.6,7.4,7.6,7.4,6.8,0016A,"(41.14972,1.17889)"
3,7.8,7.8,7.8,7.8,7.8,7.8,7.8,7.8,7.8,8.2,...,8.6,8.2,7.8,7.8,7.8,7.8,7.8,7.8,0042Y,"(41.12389,1.24917)"
4,7.0,7.0,7.0,5.6,6.8,6.8,7.2,7.2,6.8,7.2,...,7.6,6.6,7.2,7.2,6.8,7.2,7.0,6.8,0061X,"(41.41694,1.51917)"


## Looking for the nearest weather station

Now that we have the coordinates of both our destinations and all weather stations it's just a matter of crossing the two.

For some reason I can't go on without exporting and re-importing my dataframes, **Pandas** simply won't recognize the columns, and I also had to fill in the NaNs and change column names. I still don't know why.

In [35]:
#Filling missing values.

df = df.fillna(0)

In [36]:
#Changing column names.

for i in range(1,53):
    df.columns.values[i] = 'w' + str(i+1)
    
for i in range(1,53):
    meteo.columns.values[i] = 'w' + str(i+1)

In [39]:
meteo.to_csv('meteo.csv', index=False)
meteo = pd.read_csv('meteo.csv')

df.to_csv('df.csv', index=False)
df = pd.read_csv('df.csv')

In [43]:
dist_list = []

for i in range(len(towns)):
    distance = 2000
    index = 'p'
    for n in range(len(meteo)):
        d = hs.haversine(eval(towns['coords'].iloc[i]), eval(meteo['coords'].iloc[n]))
        if d < distance:
            distance = d
            index = n
    for p in range(1,54): #Iterating through all column (week) numbers.
        val = 'w' + str(p)
        df[val].iloc[i] = meteo[val].iloc[index]

C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [51]:
df.head()

,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,...,w47,w48,w49,w50,w51,w52,w53,coords,town_ID,CCAA_ID
0,7.8,7.4,7.8,7.8,7.8,7.8,7.8,7.8,7.8,8.2,...,8.2,7.8,7.8,7.8,7.8,7.8,7.8,"(41.38424664,2.17634927)",884,9
1,8.2,7.8,7.8,8.2,8.2,7.8,7.8,7.8,8.6,9.0,...,8.2,8.2,7.8,8.2,8.2,8.2,7.8,"(39.47534441,-0.37565717)",7257,10
2,8.6,8.6,8.6,9.0,8.4,8.6,8.2,8.6,8.8,9.0,...,7.8,8.2,9.0,9.0,9.0,9.0,8.2,"(36.72034267,-4.41997511)",4547,1
3,8.2,7.8,7.8,8.0,8.4,8.4,8.6,8.6,8.8,8.8,...,9.0,8.2,8.2,7.8,8.2,8.2,7.6,"(37.98436361,-1.1285408)",4613,14
4,7.8,7.8,7.8,8.2,8.2,8.2,7.8,7.8,8.6,9.0,...,8.6,8.2,8.2,8.2,8.2,8.2,7.8,"(38.34548705,-0.4831832)",151,10


In [49]:
#Typecasting CCAA_ID.

df["CCAA_ID"] = df["CCAA_ID"].astype(int)

In [50]:
#Exporting our results.

df.to_csv('weekly_weather.csv', index=False)

# Mapping all weather stations

We can easily use **Folium** to map all our weather stations on a map and visually inspect it.

In [54]:
map = folium.Map(location=[40.4167, -3.70325], zoom_start=6, tiles="OpenStreetMap") #Initiating our map.

for i in range(len(meteo)):
    folium.Marker(eval(meteo['coords'].iloc[i])).add_to(map)
    
map #Displaying the map.

# Creating the API call response

It's time to finally create our API response, keeping in mind that it must meet our criteria (point 1 of the notebook).

In [74]:
#Creating a view of a given week and other valuable columns sorted by CWI.

df[['w28', 'town_ID', 'CCAA_ID']].sort_values('w28', ascending=False)

,w28,town_ID,CCAA_ID
172,9.6,5067,3
73,9.6,3165,16
54,9.6,3219,16
139,9.4,5834,6
131,9.4,4251,12
...,...,...,...
37,6.6,3603,1
183,6.2,2731,1
78,6.2,2771,1
96,6.0,2747,1


In [118]:
#Filtering by CCAA.

df[['w1', 'town_ID', 'CCAA_ID']][df['CCAA_ID'] == 6].sort_values('w1', ascending=False).head(1)['town_ID'].tolist()[0]

5832

In [141]:
#Defining our final function:

def API_response(month, day, ccaa_list):
    """
    Input: month, day, a list of all CCAA to choose from.
    
    Output: 5 random locations with the best climate in that week, randomly split between all CCAA,
    """
    weekNumber = str(datetime.date(2021, month, day).isocalendar()[1])
    response = [df[['w'+weekNumber, 'town_ID', 'CCAA_ID']][df['CCAA_ID'] == random.choice(ccaa_list)].sort_values('w'+weekNumber, ascending=False).head(1)['town_ID'].tolist()[0], df[['w'+weekNumber, 'town_ID', 'CCAA_ID']][df['CCAA_ID'] == random.choice(ccaa_list)].sort_values('w'+weekNumber, ascending=False).head(1)['town_ID'].tolist()[0], df[['w'+weekNumber, 'town_ID', 'CCAA_ID']][df['CCAA_ID'] == random.choice(ccaa_list)].sort_values('w'+weekNumber, ascending=False).head(1)['town_ID'].tolist()[0], df[['w'+weekNumber, 'town_ID', 'CCAA_ID']][df['CCAA_ID'] == random.choice(ccaa_list)].sort_values('w'+weekNumber, ascending=False).head(1)['town_ID'].tolist()[0], df[['w'+weekNumber, 'town_ID', 'CCAA_ID']][df['CCAA_ID'] == random.choice(ccaa_list)].sort_values('w'+weekNumber, ascending=False).head(1)['town_ID'].tolist()[0]] 
    return response

In [142]:
#Testing our function.

API_response(6,15, [1,3,6])

[5832, 5067, 2800, 5832, 5832]

API up and running on local, migrating to server right now.

http://127.0.0.1:5000/api/weather?mm=6&dd=15&ccaa=[1,3,6]